In [1]:
# %pip install -q rasterio
# %pip install -q rio-cogeo
# %pip install -q owslib
# %pip install -q rioxarray
# %pip install -q geopandas
# %pip install -q folium

In [2]:
from maap.maap import MAAP
maap = MAAP()
import json
from IPython.display import display, Image
import ipycmc
w = ipycmc.MapCMC()
from pprint import pprint
import rasterio as rio
from rasterio.plot import show
import requests
import os
from owslib.wcs import WebCoverageService
import rioxarray
import warnings
warnings.filterwarnings('ignore')
import geopandas as gpd
import folium

# MAAP Version 1 Delivery 2 (V1D2) Demo Notebook

## Responding to the Usability Study and User Working Group Needs

### What were the the recommendations from the MAAP Usability Study?

* Create additional documentation
* Allow users to visualize data on a map
* Help users deal with large amounts of data

### How did the data system respond? (Demo Outline)

For V1D2, the MAAP data system made updates to address the recommendations of the usability study. This notebook demonstrates those updates. In it we:

* Demonstrate using the **[documenation site](https://maap-project.readthedocs.io/en/latest/)** to:
  - query by additional attributes
  - visualize data on a map
* Demonstrate users can handle **large amounts of data** using cloud-optimized formats and OGC standards with:
    * NASA Shuttle Radar Topography Mission Global 1 arc second V003 (COGs)
      - **Visualization** - WMTS with the Dynamic Tiler API + MAAP API
    * AfriSAR UAVSAR Coregistered SLCs Generated Using NISAR Tools (COGs)
      - **Query** - WCS via EDAV
    * ATLAS/ICESat-2 L3A Land and Vegetation Height V003 (EPT)
      - **Visualization** - 3D Tiles 
      - **Query** - Features
 

# Using Documentation

We can use [Searching by Additional Attributes](https://maap-project.readthedocs.io/en/latest/search/granules.html?highlight=additional#Searching-by-Additional-Attributes) documentation to search by the requested additional attributes `direction` and `polarization`.

In [3]:
results = maap.searchGranule(orbit_dir = "ASCENDING", limit=100)
pprint(f'Got {len(results)} results')
# print(json.dumps(results[0], indent=2))

'Got 100 results'


In [4]:
results = maap.searchGranule(polarization = "HH", limit=100)
pprint(f'Got {len(results)} results')
# print(json.dumps(results[0], indent=2))

'Got 100 results'


We can use [Visualizing Web Map Tile Service (WMTS) Layers
](https://maap-project.readthedocs.io/en/latest/visualization/using_pycmc.html) documentation to visualize data on a map.

In [5]:
# Import the ipycmc module
import ipycmc
# utilize the CMC widget
w = ipycmc.MapCMC()
w

MapCMC()

In [6]:
w.load_layer_config("https://api.maap-project.org/api/wmts/GetCapabilities", "wmts/xml")

# Handling Large Amounts of Data

## SRTM Cloud-Optimized GeoTiffs (COGs)

SRTM (Shuttle Radar Topography Mission) obtained elevation data on a near-global scale using radar interferometry. The SRTMGL1 contains elevation data in 1° X 1° tiles at 1 arc second (about 30 meters) resolution.

During this delivery, SRTMGL1 data product was made available in **Cloud-Optimized GeoTiff (COG)** format so that it can be dynammically visualizaed on a MAAP alongside the previoously published COGs of UAVSAR and LVIS.

## SRTM Visualization

In [7]:
results = maap.searchGranule(short_name="SRTMGL1_COD", bounding_box="7.27,-5.24,17.00,3.48")
for r in results:
    granule_ur = r['Granule']['GranuleUR']
    w.load_layer_config(f"https://api.maap-project.org/api/wmts/GetCapabilities?granule_ur={granule_ur}", "wmts/xml")

## Querying UAVSAR AfriSAR using WCS

Let's send a request to the EDAV WCS and extract a subset of the data in GeoTiff format and save it to our workspace.

In [8]:
c = maap.searchCollection(short_name="AfriSAR_UAVSAR_Coreg_SLC")[0]
c['Collection']['Description']

'This dataset contains multi-baseline Polarimetric Interferometric Synthetic Aperture Radar SLC (single-look-complex) data collected from multiple repeat-pass flights over Gabonese forests using the Uninhabited Aerial Vehicle Synthetic Aperture Radar (UAVSAR) instrument in February-March 2016. Supplementary data products based on various intermediate parameters of the UAVSAR data are provided and include viewing and terrain geometry.'

In [ ]:
# Configure the WCS source
EDAV_WCS_Base = "https://edav-wcs.adamplatform.eu/wcs"
wcs = WebCoverageService(f'{EDAV_WCS_Base}?service=WCS', version='2.0.0')

# Request the data from WCS
response = wcs.getCoverage(
    identifier=['uavsar_AfriSAR_v1_SLC'],
    format='image/tiff',
    filter='false',
    scale=1,
    subsets=[('Long',11.6,11.7),('Lat',-0.2,-0.1)]
)

# Save the results to file as a tif
results = "EDAV_example.tif"
with open(results, 'wb') as file:
    file.write(response.read())

Do a quick check that the data is valid, and contains spatial metadata. For fun we can check if it's a Cloud Optimized Geotiff.

In [ ]:
!gdalinfo {results}
!rio cogeo validate {results}

### Read the data and do some quick visual exploration.

In [ ]:
edav_x = rioxarray.open_rasterio(results)
edav_x.plot(cmap="gist_earth", figsize=(10, 8)).set_clim(0, 0.4)

# Handling Large Amounts of (Point Cloud) Data (Continued)

## ATL08 v003 Entwine Point Tiles (EPT)

During V1D2, the data team made the ATL08 IceSAT-2 Dataset available in [Entwine Point Tile (EPT)](https://entwine.io/entwine-point-tile.html) format.

### What are Entwine Point Tiles?

Entwine Point Tiles are a cloud-optimized octree data format for storing and visualizing massive point clouds efficiently. This format is gaining a lot of momentum and interest with an active development community.


### Visualizing with potree

Potree is a tool for visualizing EPT stores directly.

https://potree.entwine.io/data/view.html?r=%22https://cumulus-map-internal.s3.amazonaws.com/file-staging/nasa-map/ATL08_ARD-beta___001/global/ept%22

In [ ]:
img_src = "images/Oct-28-2020 16-41-44.gif"
Image(url = img_src)

### Visualizing with the 3D Tiles Service (OGC)

Cesium is a 3D tool which can be used to visualize point clouds alongside 2D data using a 3D Tiles Service.

* [3D Tiles API Endpoint](https://api.maap.xyz/api/3d-tiles/ATL08_ARD-beta___001/global/ept/ept-tileset/tileset.json)
* [Demo using Cesium](http://cesium.entwine.io/?url=https://api.maap.xyz/api/3d-tiles/ATL08_ARD-beta___001/global/ept/ept-tileset/tileset.json)

In [ ]:
img_src = "images/Oct-28-2020 16-53-01.gif"
Image(url = img_src)

### Visualizing with ipyCMC

In [ ]:
w.load_layer_config("https://cmr.maap-project.org/search/concepts/G1200354094-NASA_MAAP.json", "json", {"handleAs": "vector-3d-tile"})

## Querying ATL08 EPT using the Features Service

We can use the features service for some basic querying of the EPT Store.

### Query By Bounding Box

In [ ]:
# Format a request to the API
api_url = "https://obnrh8ozt0.execute-api.us-east-2.amazonaws.com/collections/Global/items"

# bbox should be defined as xmin, xmax, (min value z), ymin, ymax, (max value z)
# Make a request for a bounding box over Peru
bbox="-77,-26,300,-73,0,500"

payload = {
    "f": "json",
    "limit": 100,
    "bbox": bbox,
}

r = requests.get(api_url, params = payload)

In [ ]:
# Get the results directly into a Geo Data Frame (saving to file not required but recommended)
api_geojson = r.json()
api_geojson.keys()
adf = gpd.GeoDataFrame.from_features(api_geojson["features"], crs='epsg:4326')
adf.head()

In [ ]:
m = folium.Map(
    location=[adf.centroid[0].y, adf.centroid[0].x],
    zoom_start=10,
    tiles='Stamen Terrain'
)


folium.GeoJson(
    adf,
    name = "geojson"
).add_to(m)

m

### Query by Granule Id

In [ ]:
granule_id = 'ATL08_20181014035224_02370107_003_01'
payload = {
    "f": "json",
    "origin": granule_id,
}

r = requests.get(api_url, params = payload)
api_geojson = r.json()
api_geojson.keys()
adf = gpd.GeoDataFrame.from_features(api_geojson["features"], crs='epsg:4326')
adf.head()

## More query options with pdal

The [documentation](https://maap-project.readthedocs.io/en/latest/query/testing-ept-stores.html#PDAL-Pipelines) provides additional options for how to query with PDAL (Point Cloud Data Abstraction Library).

# Thank You!

**Big thanks to** Aaron Kaulfus, Alex Mandel, Chuck Daniels, David Bitner, Hai, Kaylin Bugbee, Slesa, Sam Ayers and Seth Vincent with whom none of this would be possible.